# 1. Import MEWpy packages and Human1 model (2020 version used)

In [ ]:
from mewpy.simulation import solvers
from mewpy.simulation import set_default_solver
set_default_solver('glpk')

In [2]:
from cobra.io.sbml import read_sbml_model
model = read_sbml_model('Human-GEM-annotated.xml')
model

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


Name,HumanGEM
Memory address,0x01206acf70
Number of metabolites,8400
Number of reactions,13096
Number of groups,143
Objective expression,1.0*biomass_human - 1.0*biomass_human_reverse_fb2f2
Compartments,"Cytosol, Extracellular, Lysosome, Endoplasmic reticulum, Mitochondria, Peroxisome, Golgi apparatus, Nucleus, Inner mitochondria"


# 2. Apply media conditions, and organise expression values ready for integration

## 2a. Import pre-defined media dictionary (see media notebooks)

In [3]:
import pandas as pd
%store -r dmem_extra_glut_10_FBS_media
media = dmem_extra_glut_10_FBS_media
media

{'HMR_9066': (-0.084, 1000),
 'HMR_9065': (-0.0626, 1000),
 'HMR_9063': (-0.876, 1000),
 'HMR_9067': (-0.03, 1000),
 'HMR_9038': (-0.042, 1000),
 'HMR_9039': (-0.105, 1000),
 'HMR_9040': (-0.105, 1000),
 'HMR_9041': (-0.146, 1000),
 'HMR_9042': (-0.03, 1000),
 'HMR_9043': (-0.066, 1000),
 'HMR_9069': (-0.042, 1000),
 'HMR_9044': (-0.095, 1000),
 'HMR_9045': (-0.016, 1000),
 'HMR_9064': (-0.10379, 1000),
 'HMR_9046': (-0.094, 1000),
 'HMR_9146': (-0.004, 1000),
 'HMR_9361': (-0.0072, 1000),
 'HMR_9378': (-0.004, 1000),
 'HMR_9145': (-0.004, 1000),
 'HMR_9144': (-0.004, 1000),
 'HMR_9143': (-0.0004, 1000),
 'HMR_9159': (-0.004, 1000),
 'HMR_9034': (-4.5, 1000),
 'EX_pail_hs[e]': (-1000, 1000),
 'HMR_7108': (0, 1000),
 'HMR_7110': (0, 1000),
 'HMR_7112': (0, 1000),
 'HMR_7114': (0, 1000),
 'HMR_7116': (0, 1000),
 'HMR_7118': (0, 1000),
 'HMR_7120': (0, 1000),
 'HMR_7122': (0, 1000),
 'HMR_7124': (0, 1000),
 'HMR_7126': (0, 1000),
 'HMR_9023': (0, 1000),
 'HMR_9024': (0, 1000),
 'HMR_9032'

## 2b. Use MEWpy get_simulator function to define media conditions in the 'envcond' argument

In [4]:
from mewpy.simulation import get_simulator
media_only_simulation = get_simulator(model, envcond = media)
media_only_result = media_only_simulation.simulate()
print(media_only_result)

objective: 0.21377181121421862
Status: OPTIMAL
Constraints: OrderedDict([('HMR_9066', (-0.084, 1000)), ('HMR_9065', (-0.0626, 1000)), ('HMR_9063', (-0.876, 1000)), ('HMR_9067', (-0.03, 1000)), ('HMR_9038', (-0.042, 1000)), ('HMR_9039', (-0.105, 1000)), ('HMR_9040', (-0.105, 1000)), ('HMR_9041', (-0.146, 1000)), ('HMR_9042', (-0.03, 1000)), ('HMR_9043', (-0.066, 1000)), ('HMR_9069', (-0.042, 1000)), ('HMR_9044', (-0.095, 1000)), ('HMR_9045', (-0.016, 1000)), ('HMR_9064', (-0.10379, 1000)), ('HMR_9046', (-0.094, 1000)), ('HMR_9146', (-0.004, 1000)), ('HMR_9361', (-0.0072, 1000)), ('HMR_9378', (-0.004, 1000)), ('HMR_9145', (-0.004, 1000)), ('HMR_9144', (-0.004, 1000)), ('HMR_9143', (-0.0004, 1000)), ('HMR_9159', (-0.004, 1000)), ('HMR_9034', (-4.5, 1000)), ('EX_pail_hs[e]', (-1000, 1000)), ('HMR_7108', (0, 1000)), ('HMR_7110', (0, 1000)), ('HMR_7112', (0, 1000)), ('HMR_7114', (0, 1000)), ('HMR_7116', (0, 1000)), ('HMR_7118', (0, 1000)), ('HMR_7120', (0, 1000)), ('HMR_7122', (0, 1000)), ('

## 2c. Find media-constrained biomass production rate, and essential reactions/genes

In [5]:
media_only_result.find(['biomass_human'])

,Flux rate
Reaction ID,
biomass_human,0.213772


In [6]:
dmem_extra_glut_10_FBS_essential_reactions = media_only_simulation.essential_reactions()
dmem_extra_glut_10_FBS_essential_genes = media_only_simulation.essential_genes()

%store -r dmem_extra_glut_10_FBS_essential_reactions
%store -r dmem_extra_glut_10_FBS_essential_genes

In [7]:
#set standard essential reactions variable,
#which can be used across model notebooks with different medias
media_only_essential_reactions = dmem_extra_glut_10_FBS_essential_reactions
media_only_essential_genes = dmem_extra_glut_10_FBS_essential_genes

## 2d. Import and organise normalised transcriptomics data

In [8]:
#normalised CCLE transcriptomics (TPM, RSEM, Log2-tranformed with pseudo-count of 1 to avoid negative values)
#details for dataset PMID: 22460905
%store -r t
t

,TSPAN6 (ENSG00000000003),TNMD (ENSG00000000005),DPM1 (ENSG00000000419),SCYL3 (ENSG00000000457),C1orf112 (ENSG00000000460),FGR (ENSG00000000938),CFH (ENSG00000000971),FUCA2 (ENSG00000001036),GCLC (ENSG00000001084),NFYA (ENSG00000001167),...,ENSG00000288714,ENSG00000288717,ENSG00000288718,ENSG00000288719,ENSG00000288720,ENSG00000288721,ENSG00000288722,ENSG00000288723,ENSG00000288724,ENSG00000288725
cell_line,,,,,,,,,,,,,,,,,,,,,
COV434_OVARY,2.946731,0.275007,6.527321,1.761285,3.160275,0.000000,5.204376,0.464668,3.044394,3.435629,...,0.411426,0.0,0.028569,0.014355,0.042644,0.757023,1.925999,0.000000,0.0,0.000000
59M_OVARY,3.460743,0.000000,6.399000,1.855990,3.374344,0.028569,3.261531,6.094869,5.290203,3.947666,...,0.028569,0.0,0.137504,0.000000,0.056584,0.978196,3.116032,0.000000,0.0,0.137504
NZOV9_OVARY,6.017922,0.000000,6.544578,3.047887,4.425594,0.000000,0.000000,4.412104,5.500165,4.053111,...,0.000000,0.0,0.739848,0.028569,0.056584,0.275007,2.443607,0.000000,0.0,0.000000
OAW42_OVARY,4.318317,0.000000,7.045377,1.831877,4.016140,0.014355,0.042644,6.321567,3.739848,4.559492,...,0.000000,0.0,0.056584,0.000000,0.000000,0.150560,2.929791,0.028569,0.0,0.000000
COV644_OVARY,4.152995,0.000000,6.044394,2.104337,3.852998,0.201634,0.773996,6.676662,3.993674,4.136684,...,0.000000,0.0,0.124328,0.056584,0.111031,0.443607,4.688740,0.757023,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RMUGS_OVARY,4.426265,0.000000,5.912650,2.114367,4.017031,0.000000,1.594549,5.535742,6.410748,3.005400,...,0.000000,0.0,0.028569,0.000000,0.150560,0.831877,4.978196,0.028569,0.0,0.070389
HEYA8_OVARY,3.718088,0.000000,7.133810,1.922198,3.787641,0.000000,0.475085,6.115824,4.336997,2.946731,...,0.000000,0.0,0.855990,0.014355,0.042644,0.575312,4.095924,0.000000,0.0,0.000000
PEO4_OVARY,4.288359,0.000000,6.011451,1.111031,2.735522,0.000000,1.704872,5.599913,2.773996,2.553361,...,0.000000,0.0,0.056584,0.000000,0.000000,0.097611,4.644433,0.000000,0.0,0.000000


In [9]:
#this chunk of code can be personalised for your dataframe. Just want gene IDs which match your model's
#gene-reaction rules, and corresponding normalised gene expression abundances

#access row with cell line of choice. this row contains expression values
gene_exp = t.iloc[1,:]

#save column headers (keys) from above 'gene_exp' row into list
g_list = []
for k in gene_exp.keys():
    g_list.append(k)

#for each string in list from above, where both gene name and ensembl ID are in string, 
#save only ensembl ID to new list ('g_ensembl')
#where only ensembl ID is in string, save this to 'g_ensembl' list
g_ensembl = []
for n in range(len(g_list)):
    if 'ENS' not in (g_list[n].split())[0]:
        g_ensembl.append((g_list[n].split())[1])
    if 'ENS' in (g_list[n].split())[0]:
        g_ensembl.append((g_list[n].split())[0])
        
#remove punctuation from list, as some brackets are present in original dataframe
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
g_ensembl_2 = []
for n in range(len(g_ensembl)):
    for x in g_ensembl[n]:
        if x in punc:
            g_ensembl[n] = g_ensembl[n].replace(x, "")
    g_ensembl_2.append(g_ensembl[n])
g_ensembl_2

#save list of expression values for cell line of choice into list format 'vs'
vs = []
for v in gene_exp:
    vs.append(v)
    
#create dictionary with formated ensembl ids (from 'g_ensembl_2' list), 
#and corresponding expression value (from 'vs' list)
gene_exp_dict = {}
for n in range(len(g_ensembl_2)):
    gene_exp_dict[g_ensembl_2[n]] = vs[n]
gene_exp_dict

{'ENSG00000000003': 3.460742563789644,
 'ENSG00000000005': 0.0,
 'ENSG00000000419': 6.399000148261131,
 'ENSG00000000457': 1.8559896973084808,
 'ENSG00000000460': 3.374343989038501,
 'ENSG00000000938': 0.0285691521967709,
 'ENSG00000000971': 3.261530815243406,
 'ENSG00000001036': 6.094869432718697,
 'ENSG00000001084': 5.29020320512607,
 'ENSG00000001167': 3.9476661567960094,
 'ENSG00000001460': 3.2265085298086795,
 'ENSG00000001461': 4.043519493662743,
 'ENSG00000001497': 4.886062338345599,
 'ENSG00000001561': 1.9708536543404835,
 'ENSG00000001617': 3.040892430646901,
 'ENSG00000001626': 0.01435529297707,
 'ENSG00000001629': 3.1127001327493624,
 'ENSG00000001630': 4.815575428862573,
 'ENSG00000001631': 2.788685710613534,
 'ENSG00000002016': 3.1618876823768938,
 'ENSG00000002079': 0.1243281350022017,
 'ENSG00000002330': 5.986183362881372,
 'ENSG00000002549': 5.593353770980297,
 'ENSG00000002586': 7.654206377944292,
 'ENSG00000002587': 0.0565835283663675,
 'ENSG00000002726': 0.0285691521

In [10]:
_59m_gene_exp_dict = gene_exp_dict #save expression dictionary as unique cell line variable
%store _59m_gene_exp_dict

Stored '_59m_gene_exp_dict' (dict)


# 3. Organise genome-scale model according to gene-reaction rules

## 3a. 'and' & 'or' terms etc to iterate through later on

In [11]:
#divide rules into different categories, depending on enzyme requirement
ANDs = [] #enzyme subunits
ORs = [] #isoenzymes
ANDORs = [] #both subunits and isoenzymes
one_gene = [] #single enzyme 
no_gene = [] #no enzyme, e.g. simple diffusion, or ion-channel...

for r in model.reactions:
    if 'and' in r.gene_reaction_rule and 'or' not in r.gene_reaction_rule:
        ANDs.append(r.id)
    if 'and' in r.gene_reaction_rule and 'or' in r.gene_reaction_rule:
        ANDORs.append(r.id)
    if 'or' in r.gene_reaction_rule and 'and' not in r.gene_reaction_rule:
        ORs.append(r.id)
    if len(r.gene_reaction_rule) == 0:
        no_gene.append(r.id)
    if len(r.gene_reaction_rule) != 0:
        if 'or' in r.gene_reaction_rule:
            continue
        elif 'and' in r.gene_reaction_rule:
            continue
        else:
            one_gene.append(r.id)

print('AND rules: ', len(ANDs))
print('ANDOR rules: ', len(ANDORs))
print('OR rules: ', len(ORs))
print('ONE GENE rules: ', len(one_gene))
print('NO GENE rules: ', len(no_gene))
print('Proportion of model not annotated: ', len(no_gene)/len(model.reactions)*100, '%')
print('Proportion of model which IS annotated: ', 100-(len(no_gene)/len(model.reactions)*100), '%')
print('Total Reactions = ', len(model.reactions))
print(len(ORs)+len(ANDs)+len(ANDORs)+len(one_gene)+len(no_gene))

AND rules:  653
ANDOR rules:  129
OR rules:  3972
ONE GENE rules:  3282
NO GENE rules:  5060
Proportion of model not annotated:  38.63775198533904 %
Proportion of model which IS annotated:  61.36224801466096 %
Total Reactions =  13096
13096


# 4. Integration of expression data into reactions bounds

## 4a. Organise 'one-gene' rules, according to reversibility

In [12]:
one_gene_forward = []
one_gene_reversible = []
for r in one_gene:
    if model.reactions.get_by_id(r).reversibility == True:
        one_gene_reversible.append(r)
    else:
        one_gene_forward.append(r)
print('number of reversible, one gene reactions =', len(one_gene_reversible))
print('number of forward, one gene reactions =', len(one_gene_forward))

number of reversible, one gene reactions = 1074
number of forward, one gene reactions = 2208


In [13]:
gene_exp_dict = _59m_gene_exp_dict

## 4b. Integrate data to 'one-gene' rules, forward direction

In [14]:
one_gene_constrained_reactions = {} #create dictionary to contain reaction bounds, according to code below
                                    #this dictionary will be used by MEWpy to constrain model 
for r in one_gene_forward:
    if r not in media.keys(): #do not overwrite media conditions
        if r not in media_only_essential_reactions: #do not constrain essential reactions/genes, or growth will be blocked
            if model.reactions.get_by_id(r).gene_reaction_rule not in media_only_essential_genes:
                if model.reactions.get_by_id(r).gene_reaction_rule in gene_exp_dict.keys(): #check we have expression data
                    if gene_exp_dict[model.reactions.get_by_id(r).gene_reaction_rule] != (0,0):
                        model.reactions.get_by_id(r).bounds = (0,(gene_exp_dict[model.reactions.get_by_id(r).gene_reaction_rule])) #set upper bound equal to expression value
                        solution = model.optimize() #optimise after constraint, using COBRApy
                        if solution.fluxes['biomass_human'] < 0.0208: #enforce experimental growth value as minimum gorwth rates constraints can reach
                            print(r, ':', 'constrained bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                            model.reactions.get_by_id(r).bounds = (0,1000) #if growth is constrained below experimental, then reopen reaction
                            solution = model.optimize()
                            print(r, ':', 're-opened bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                            one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds #save reaction bounds to dictionary
                        if solution.fluxes['biomass_human'] !=0:
                            if solution.fluxes['biomass_human'] > 0.0208:
                                print(r, ':', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                                one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds #use expression data as reaction bound, if it doesn't fall below experimental

HMR_3907 : (0, 6.73226919950145) 0.21377181121421862
HMR_4097 : (0, 4.114367024952) 0.21377181121421862
HMR_4108 : (0, 4.114367024952) 0.21377181121421862
HMR_4133 : (0, 4.114367024952) 0.21377181121421862
HMR_4360 : (0, 2.4672794804599825) 0.21377181121421862
HMR_4372 : (0, 3.2387868595871163) 0.21377181121421862
HMR_7747 : (0, 4.472487771462744) 0.21377181121421862
HMR_8360 : (0, 5.22881869049588) 0.21377181121421862
HMR_8757 : (0, 0.0976107966264223) 0.21377181121421862
HMR_5397 : (0, 5.842978831788326) 0.21377181121421862
HMR_5399 : (0, 1.6780719051126378) 0.21377181121421862
HMR_5400 : (0, 1.6780719051126378) 0.21377181121421862
HMR_8592 : (0, 4.049630767724601) 0.21377181121421862
HMR_8589 : (0, 4.049630767724601) 0.21377181121421862
HMR_8583 : (0, 0.0426443374084937) 0.21377181121421862
HMR_8584 : (0, 0.0) 0.21377181121421862
HMR_8585 : (0, 1.2265085298086795) 0.21377181121421862
HMR_3944 : (0, 5.443938115013176) 0.21377181121421862
HMR_4132 : (0, 3.839959587489532) 0.2137718112

In [15]:
%store one_gene_constrained_reactions

Stored 'one_gene_constrained_reactions' (dict)


In [16]:
one_gene_forward_simulation = get_simulator(model, envcond = media, constraints = one_gene_constrained_reactions) #one_gene_constrained_reactions
one_gene_forward_result = one_gene_forward_simulation.simulate()
print(one_gene_forward_result) #use MEWpy to add media and expression constraints onto model, then print biomass

objective: 0.0958684638238402
Status: OPTIMAL
Constraints: OrderedDict([('HMR_9066', (-0.084, 1000)), ('HMR_9065', (-0.0626, 1000)), ('HMR_9063', (-0.876, 1000)), ('HMR_9067', (-0.03, 1000)), ('HMR_9038', (-0.042, 1000)), ('HMR_9039', (-0.105, 1000)), ('HMR_9040', (-0.105, 1000)), ('HMR_9041', (-0.146, 1000)), ('HMR_9042', (-0.03, 1000)), ('HMR_9043', (-0.066, 1000)), ('HMR_9069', (-0.042, 1000)), ('HMR_9044', (-0.095, 1000)), ('HMR_9045', (-0.016, 1000)), ('HMR_9064', (-0.10379, 1000)), ('HMR_9046', (-0.094, 1000)), ('HMR_9146', (-0.004, 1000)), ('HMR_9361', (-0.0072, 1000)), ('HMR_9378', (-0.004, 1000)), ('HMR_9145', (-0.004, 1000)), ('HMR_9144', (-0.004, 1000)), ('HMR_9143', (-0.0004, 1000)), ('HMR_9159', (-0.004, 1000)), ('HMR_9034', (-4.5, 1000)), ('EX_pail_hs[e]', (-1000, 1000)), ('HMR_7108', (0, 1000)), ('HMR_7110', (0, 1000)), ('HMR_7112', (0, 1000)), ('HMR_7114', (0, 1000)), ('HMR_7116', (0, 1000)), ('HMR_7118', (0, 1000)), ('HMR_7120', (0, 1000)), ('HMR_7122', (0, 1000)), ('H

In [17]:
one_gene_forward_result.find(['biomass_human'])

,Flux rate
Reaction ID,
biomass_human,0.095868


In [18]:
one_gene_forward_essential_reactions = media_only_essential_reactions
one_gene_forward_essential_genes = media_only_essential_genes

## 4c. 'one-gene' rules, reversible

In [19]:
for r in one_gene_reversible: #run constraints code for reversible, one-gene reactions
    if r not in media.keys():
        if r not in one_gene_forward_essential_reactions: #change to new dict
            if model.reactions.get_by_id(r).gene_reaction_rule not in one_gene_forward_essential_genes:
                if model.reactions.get_by_id(r).gene_reaction_rule in gene_exp_dict.keys(): 
                    if gene_exp_dict[model.reactions.get_by_id(r).gene_reaction_rule] != (0,0): 
                        #here, set lower bound as -1*expression value, and upper bound as positive expression value
                        model.reactions.get_by_id(r).bounds = (-1*(gene_exp_dict[model.reactions.get_by_id(r).gene_reaction_rule]),(gene_exp_dict[model.reactions.get_by_id(r).gene_reaction_rule]))
                        solution = model.optimize()
                        if solution.fluxes['biomass_human'] < 0.0208:
                            print(r, ':', 'constrained bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                            model.reactions.get_by_id(r).bounds = (-1000,1000)
                            solution = model.optimize()
                            print(r, ':', 're-opened bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                            #update constraints dictionary with reversible reaction bounds
                            one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
                        if solution.fluxes['biomass_human'] !=0:
                            if solution.fluxes['biomass_human'] > 0.0208:
                                print(r, ':', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                                one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds

HMR_4381 : (-8.189972773359631, 8.189972773359631) 0.09586846382384082
HMR_4391 : (-9.776219162140023, 9.776219162140023) 0.09586846382384082
HMR_7745 : (-3.196921733820329, 3.196921733820329) 0.09586846382384082
HMR_7748 : (-8.189972773359631, 8.189972773359631) 0.09586846382384082
HMR_7749 : (-8.189972773359631, 8.189972773359631) 0.09586846382384082
HMR_4128 : (-6.745775149485796, 6.745775149485796) 0.09586846382384082
HMR_4131 : (-3.839959587489532, 3.839959587489532) 0.09586846382384082
HMR_4315 : (-3.902073579310743, 3.902073579310743) 0.09586846382384082
HMR_4383 : (-3.778208576398088, 3.778208576398088) 0.09586846382384082
HMR_4402 : (-2.0806576633452254, 2.0806576633452254) 0.09586846382384082
HMR_4595 : (-1.7092906357233577, 1.7092906357233577) 0.09586846382384082
HMR_8344 : (-6.73226919950145, 6.73226919950145) 0.09586846382384082
HMR_8352 : (-2.6158870739160567, 2.6158870739160567) 0.09586846382384082
HMR_6537 : (-6.73226919950145, 6.73226919950145) 0.09586846382384082
HMR_

In [20]:
all_one_gene_simulation = get_simulator(model, envcond = media, constraints = one_gene_constrained_reactions) #one_gene_constrained_reactions
all_one_gene_result = all_one_gene_simulation.simulate()
print(all_one_gene_result)

objective: 0.07902460901893985
Status: OPTIMAL
Constraints: OrderedDict([('HMR_9066', (-0.084, 1000)), ('HMR_9065', (-0.0626, 1000)), ('HMR_9063', (-0.876, 1000)), ('HMR_9067', (-0.03, 1000)), ('HMR_9038', (-0.042, 1000)), ('HMR_9039', (-0.105, 1000)), ('HMR_9040', (-0.105, 1000)), ('HMR_9041', (-0.146, 1000)), ('HMR_9042', (-0.03, 1000)), ('HMR_9043', (-0.066, 1000)), ('HMR_9069', (-0.042, 1000)), ('HMR_9044', (-0.095, 1000)), ('HMR_9045', (-0.016, 1000)), ('HMR_9064', (-0.10379, 1000)), ('HMR_9046', (-0.094, 1000)), ('HMR_9146', (-0.004, 1000)), ('HMR_9361', (-0.0072, 1000)), ('HMR_9378', (-0.004, 1000)), ('HMR_9145', (-0.004, 1000)), ('HMR_9144', (-0.004, 1000)), ('HMR_9143', (-0.0004, 1000)), ('HMR_9159', (-0.004, 1000)), ('HMR_9034', (-4.5, 1000)), ('EX_pail_hs[e]', (-1000, 1000)), ('HMR_7108', (0, 1000)), ('HMR_7110', (0, 1000)), ('HMR_7112', (0, 1000)), ('HMR_7114', (0, 1000)), ('HMR_7116', (0, 1000)), ('HMR_7118', (0, 1000)), ('HMR_7120', (0, 1000)), ('HMR_7122', (0, 1000)), ('

In [21]:
all_one_gene_result.find(['biomass_human'])

,Flux rate
Reaction ID,
biomass_human,0.079025


## 4d. Organise 'or' rules (isoenzymes) according to reversibility

In [22]:
or_forward = []
or_reversible = []
for r in ORs:
    if model.reactions.get_by_id(r).reversibility == True:
        or_reversible.append(r)
    else:
        or_forward.append(r)
print('number of reversible, OR reactions =', len(or_reversible))
print('number of forward, OR reactions =', len(or_forward))

number of reversible, OR reactions = 1325
number of forward, OR reactions = 2647


## 4e. Integrate data to 'or', forward direction

In [23]:
for r in or_forward:
    if r not in media.keys():
        if r not in one_gene_forward_essential_reactions:
            rule_list = [] #make list to append with terms in reaction rule
            rule = model.reactions.get_by_id(r).gene_reaction_rule
            rule_list.append(rule.split())
            rule_genes = [] #append new list with only ensembl IDs from reaction rule
            for n in rule_list:
                for num in n:
                    if 'ENS' in num:
                        rule_genes.append(num)
            #calculate sum of expressions
            genes_in_dataset = []
            for gene in rule_genes:
                if gene in gene_exp_dict.keys():
                    genes_in_dataset.append(gene)
            exp_list = []        
            for g in genes_in_dataset:
                exp_list.append(gene_exp_dict[g])
            sum_of_expressions = sum(exp_list)
            #set bounds according to sum of expressions
            model.reactions.get_by_id(r).bounds = (0,sum_of_expressions) #same as 'one-gene' method, where expression (sum) is upper bound
            solution = model.optimize()
            if solution.fluxes['biomass_human'] < 0.0208:
                print(r, ':', 'constrained bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                model.reactions.get_by_id(r).bounds = (0,1000)
                solution = model.optimize()
                print(r, ':', 're-opened bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
            if solution.fluxes['biomass_human'] !=0:
                if solution.fluxes['biomass_human'] > 0.0208:
                    print(r, ':', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                    one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds

HMR_3905 : (0, 9.609592994353976) 0.07902460901893985
HMR_4099 : (0, 4.105895940574135) 0.07902460901893985
HMR_4283 : (0, 10.718762793250395) 0.07902460901893985
HMR_8357 : (0, 26.964602985643552) 0.07902460901893985
HMR_4379 : (0, 21.56910295768853) 0.07902460901893985
HMR_4301 : (0, 24.304625134985066) 0.07902460901893985
HMR_4355 : (0, 14.568885921143984) 0.07902460901893985
HMR_4358 : (0, 18.543525017609547) 0.07902460901893985
HMR_4370 : (0, 5.982434635944214) 0.07902460901893985
HMR_4377 : (0, 0.1505596765753814) 0.07902460901893985
HMR_4394 : (0, 21.36655510559268) 0.07902460901893985
HMR_4521 : (0, 6.539003674268171) 0.07902460901893985
HMR_7746 : (0, 16.894067334129936) 0.07902460901893985
HMR_8652 : (0, 6.539003674268171) 0.07902460901893985
HMR_3989 : (0, 6.736454422408049) 0.07902460901893985
HMR_4837 : (0, 1.366923648978518) 0.07902460901893985
HMR_5395 : (0, 7.291120037153965) 0.07902460901893985
HMR_5396 : (0, 10.996633548457648) 0.07902460901893985
HMR_5398 : (0, 10.84

In [24]:
or_forward_simulation = get_simulator(model, envcond = media, constraints = one_gene_constrained_reactions) #one_gene_constrained_reactions
or_forward_result = or_forward_simulation.simulate()
print(or_forward_result)

objective: 0.07902460901893836
Status: OPTIMAL
Constraints: OrderedDict([('HMR_9066', (-0.084, 1000)), ('HMR_9065', (-0.0626, 1000)), ('HMR_9063', (-0.876, 1000)), ('HMR_9067', (-0.03, 1000)), ('HMR_9038', (-0.042, 1000)), ('HMR_9039', (-0.105, 1000)), ('HMR_9040', (-0.105, 1000)), ('HMR_9041', (-0.146, 1000)), ('HMR_9042', (-0.03, 1000)), ('HMR_9043', (-0.066, 1000)), ('HMR_9069', (-0.042, 1000)), ('HMR_9044', (-0.095, 1000)), ('HMR_9045', (-0.016, 1000)), ('HMR_9064', (-0.10379, 1000)), ('HMR_9046', (-0.094, 1000)), ('HMR_9146', (-0.004, 1000)), ('HMR_9361', (-0.0072, 1000)), ('HMR_9378', (-0.004, 1000)), ('HMR_9145', (-0.004, 1000)), ('HMR_9144', (-0.004, 1000)), ('HMR_9143', (-0.0004, 1000)), ('HMR_9159', (-0.004, 1000)), ('HMR_9034', (-4.5, 1000)), ('EX_pail_hs[e]', (-1000, 1000)), ('HMR_7108', (0, 1000)), ('HMR_7110', (0, 1000)), ('HMR_7112', (0, 1000)), ('HMR_7114', (0, 1000)), ('HMR_7116', (0, 1000)), ('HMR_7118', (0, 1000)), ('HMR_7120', (0, 1000)), ('HMR_7122', (0, 1000)), ('

In [25]:
or_forward_result.find(['biomass_human'])

,Flux rate
Reaction ID,
biomass_human,0.079025


## 4f. 'or', reversible

In [26]:
for r in or_reversible:
    if r not in media.keys():
        if r not in one_gene_forward_essential_reactions:
            #find ensembl ids in gene-reaction rules
            rule_list = []
            rule = model.reactions.get_by_id(r).gene_reaction_rule
            rule_list.append(rule.split())
            rule_genes = []
            for n in rule_list:
                for num in n:
                    if 'ENS' in num:
                        rule_genes.append(num)
            #calculate sum of expressions
            genes_in_dataset = []
            for gene in rule_genes:
                if gene in gene_exp_dict.keys():
                    genes_in_dataset.append(gene)
            exp_list = []        
            for g in genes_in_dataset:
                exp_list.append(gene_exp_dict[g])
            sum_of_expressions = sum(exp_list)
            #set bounds according to sum of expressions; same as before, use negative and positive expression (sum) as bounds
            model.reactions.get_by_id(r).bounds = (-1*sum_of_expressions,sum_of_expressions)
            solution = model.optimize()
            if solution.fluxes['biomass_human'] < 0.0208:
                print(r, ':', 'constrained bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                model.reactions.get_by_id(r).bounds = (-1000,1000)
                solution = model.optimize()
                print(r, ':', 're-opened bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
            if solution.fluxes['biomass_human'] !=0:
                if solution.fluxes['biomass_human'] > 0.0208:
                    print(r, ':', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                    one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds

HMR_4281 : (-23.508594595058444, 23.508594595058444) 0.07902460901893836
HMR_4388 : (-23.508594595058444, 23.508594595058444) 0.07902460901893836
HMR_4363 : (-20.434466694603103, 20.434466694603103) 0.07902460901893836
HMR_4365 : (-13.730570073683273, 13.730570073683273) 0.07902460901893836
HMR_4368 : (-12.405779682585322, 12.405779682585322) 0.07575683902010422
HMR_4373 : (-12.792037847316958, 12.792037847316958) 0.0757283333093149
HMR_4375 : (-14.568885921143984, 14.568885921143984) 0.0757283333093149
HMR_4774 : (-21.56910295768853, 21.56910295768853) 0.0757283333093149
HMR_4775 : (-21.56910295768853, 21.56910295768853) 0.0757283333093149
HMR_8762 : (-14.568885921143984, 14.568885921143984) 0.0757283333093149
HMR_8766 : (-9.632952315393624, 9.632952315393624) 0.0757283333093149
HMR_4316 : (-9.632952315393624, 9.632952315393624) 0.0757283333093149
HMR_4401 : (-3.6781440380613737, 3.6781440380613737) 0.0757283333093149
HMR_4592 : (-9.632952315393624, 9.632952315393624) 0.07572833330931

In [27]:
all_or_simulation = get_simulator(model, envcond = media, constraints = one_gene_constrained_reactions) #one_gene_constrained_reactions
all_or_result = all_or_simulation.simulate()
print(all_or_result)

objective: 0.07190461598881652
Status: OPTIMAL
Constraints: OrderedDict([('HMR_9066', (-0.084, 1000)), ('HMR_9065', (-0.0626, 1000)), ('HMR_9063', (-0.876, 1000)), ('HMR_9067', (-0.03, 1000)), ('HMR_9038', (-0.042, 1000)), ('HMR_9039', (-0.105, 1000)), ('HMR_9040', (-0.105, 1000)), ('HMR_9041', (-0.146, 1000)), ('HMR_9042', (-0.03, 1000)), ('HMR_9043', (-0.066, 1000)), ('HMR_9069', (-0.042, 1000)), ('HMR_9044', (-0.095, 1000)), ('HMR_9045', (-0.016, 1000)), ('HMR_9064', (-0.10379, 1000)), ('HMR_9046', (-0.094, 1000)), ('HMR_9146', (-0.004, 1000)), ('HMR_9361', (-0.0072, 1000)), ('HMR_9378', (-0.004, 1000)), ('HMR_9145', (-0.004, 1000)), ('HMR_9144', (-0.004, 1000)), ('HMR_9143', (-0.0004, 1000)), ('HMR_9159', (-0.004, 1000)), ('HMR_9034', (-4.5, 1000)), ('EX_pail_hs[e]', (-1000, 1000)), ('HMR_7108', (0, 1000)), ('HMR_7110', (0, 1000)), ('HMR_7112', (0, 1000)), ('HMR_7114', (0, 1000)), ('HMR_7116', (0, 1000)), ('HMR_7118', (0, 1000)), ('HMR_7120', (0, 1000)), ('HMR_7122', (0, 1000)), ('

In [28]:
all_or_result.find(['biomass_human'])

,Flux rate
Reaction ID,
biomass_human,0.071905


## 4g. Organise 'and' rules, according to reversibility

In [29]:
and_forward = []
and_reversible = []
for r in ANDs:
    if model.reactions.get_by_id(r).reversibility == True:
        and_reversible.append(r)
    else:
        and_forward.append(r)
print('number of reversible, AND reactions =', len(and_reversible))
print('number of forward, AND reactions =', len(and_forward))

number of reversible, AND reactions = 92
number of forward, AND reactions = 561


## 4h. Intergate data to 'and', forward direction

In [30]:
for r in and_forward:
    if r not in media.keys():
        if r not in one_gene_forward_essential_reactions:
            #find ensembl IDs in reaction rules, as performed for 'or' rules
            rule_list = []
            rule = model.reactions.get_by_id(r).gene_reaction_rule
            rule_list.append(rule.split())
            rule_genes = []
            for n in rule_list:
                for num in n:
                    if 'ENS' in num:
                        rule_genes.append(num)
            #generate list of expressions
            genes_in_dataset = []
            for gene in rule_genes:
                if gene in gene_exp_dict.keys():
                    genes_in_dataset.append(gene)
            exp_list = []        
            for g in genes_in_dataset:
                exp_list.append(gene_exp_dict[g])
            if len(exp_list) > 1:
                exp_list.sort()
                min_expression = exp_list[0]
            if len(exp_list) == 1:
                min_expression = exp_list[0] #set variable for minimum, rate-limiting subunit expression
            #use minimum expression as upper bound for forward reactions
            model.reactions.get_by_id(r).bounds = (0,min_expression)
            solution = model.optimize()
            if solution.fluxes['biomass_human'] < 0.0208:
                print(r, ':', 'constrained bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                model.reactions.get_by_id(r).bounds = (0,1000)
                solution = model.optimize()
                print(r, ':', 're-opened bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
            if solution.fluxes['biomass_human'] !=0:
                if solution.fluxes['biomass_human'] > 0.0208:
                    print(r, ':', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                    one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds

HMR_4137 : (0, 0.0) 0.07190461598881652
HMR_4415 : (0, 0.0285691521967709) 0.07190461598881652
HMR_3800 : (0, 2.6599245584023783) 0.07190461598881652
HMR_8433 : (0, 1.232660756790275) 0.07190461598881652
HMR_8434 : (0, 1.232660756790275) 0.07190461598881652
HMR_8435 : (0, 1.232660756790275) 0.07190461594133127
HMR_8436 : (0, 1.232660756790275) 0.07190461598881658
HMR_4239 : (0, 5.181897643108388) 0.07190461598881658
HMR_6978 : (0, 4.168321115739722) 0.07190461598881658
HMR_8029 : (0, 4.168321115739722) 0.07190461598881658
HMR_3164 : (0, 1.3504972470841332) 0.07190461598881658
HMR_3166 : (0, 4.625854931368057) 0.07190461598881658
HMR_3206 : (0, 4.684257705058931) 0.07190461598881658
HMR_3208 : (0, 4.096767854714689) 0.07190461598881658
HMR_6419 : (0, 2.2539892662307865) 0.07190461598881658
HMR_6421 : (0, 2.2539892662307865) 0.07190461598881658
HMR_3748 : (0, 2.2539892662307865) 0.07190461598881658
HMR_3753 : (0, 1.3504972470841332) 0.07190461598881658
HMR_3767 : (0, 2.2539892662307865) 

In [31]:
and_forward_simulation = get_simulator(model, envcond = media, constraints = one_gene_constrained_reactions) #one_gene_constrained_reactions
and_forward_result = and_forward_simulation.simulate()
print(and_forward_result)

objective: 0.03065235921522385
Status: OPTIMAL
Constraints: OrderedDict([('HMR_9066', (-0.084, 1000)), ('HMR_9065', (-0.0626, 1000)), ('HMR_9063', (-0.876, 1000)), ('HMR_9067', (-0.03, 1000)), ('HMR_9038', (-0.042, 1000)), ('HMR_9039', (-0.105, 1000)), ('HMR_9040', (-0.105, 1000)), ('HMR_9041', (-0.146, 1000)), ('HMR_9042', (-0.03, 1000)), ('HMR_9043', (-0.066, 1000)), ('HMR_9069', (-0.042, 1000)), ('HMR_9044', (-0.095, 1000)), ('HMR_9045', (-0.016, 1000)), ('HMR_9064', (-0.10379, 1000)), ('HMR_9046', (-0.094, 1000)), ('HMR_9146', (-0.004, 1000)), ('HMR_9361', (-0.0072, 1000)), ('HMR_9378', (-0.004, 1000)), ('HMR_9145', (-0.004, 1000)), ('HMR_9144', (-0.004, 1000)), ('HMR_9143', (-0.0004, 1000)), ('HMR_9159', (-0.004, 1000)), ('HMR_9034', (-4.5, 1000)), ('EX_pail_hs[e]', (-1000, 1000)), ('HMR_7108', (0, 1000)), ('HMR_7110', (0, 1000)), ('HMR_7112', (0, 1000)), ('HMR_7114', (0, 1000)), ('HMR_7116', (0, 1000)), ('HMR_7118', (0, 1000)), ('HMR_7120', (0, 1000)), ('HMR_7122', (0, 1000)), ('

In [32]:
and_forward_result.find(['biomass_human'])

,Flux rate
Reaction ID,
biomass_human,0.030652


## 4i. 'and', reversible

In [33]:
for r in and_reversible:
    if r not in media.keys():
        if r not in one_gene_forward_essential_reactions:
            rule_list = []
            rule = model.reactions.get_by_id(r).gene_reaction_rule
            rule_list.append(rule.split())
            rule_genes = []
            for n in rule_list:
                for num in n:
                    if 'ENS' in num:
                        rule_genes.append(num)
            #generate list of expressions
            genes_in_dataset = []
            for gene in rule_genes:
                if gene in gene_exp_dict.keys():
                    genes_in_dataset.append(gene)
            exp_list = []        
            for g in genes_in_dataset:
                exp_list.append(gene_exp_dict[g])
            if len(exp_list) > 1:
                exp_list.sort()
                min_expression = exp_list[0]
            if len(exp_list) == 1:
                min_expression = exp_list[0]
            #set bounds according to minimum expression; use negative and positive expression (minimum) as bounds
            model.reactions.get_by_id(r).bounds = ((-1*min_expression),min_expression)
            solution = model.optimize()
            if solution.fluxes['biomass_human'] < 0.0208:
                print(r, ':', 'constrained bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                model.reactions.get_by_id(r).bounds = (-1000,1000)
                solution = model.optimize()
                print(r, ':', 're-opened bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
            if solution.fluxes['biomass_human'] !=0:
                if solution.fluxes['biomass_human'] > 0.0208:
                    print(r, ':', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                    one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds

HMR_4832 : (-0.0285691521967709, 0.0285691521967709) 0.03065235921522385
HMR_6409 : (-1.232660756790275, 1.232660756790275) 0.03065235921522364
HMR_8437 : (-1.232660756790275, 1.232660756790275) 0.03065235921522364
HMR_6926 : (-3.82273014794452, 3.82273014794452) 0.03065235921522364
HMR_6927 : (-3.82273014794452, 3.82273014794452) 0.03065235921522364
HMR_4735 : (-1.3504972470841332, 1.3504972470841332) 0.03065235921522364
HMR_4652 : (-6.818006901986738, 6.818006901986738) 0.03065235921522364
HMR_8743 : (-6.818006901986738, 6.818006901986738) 0.03065235921522364
HMR_3454 : (-4.625854931368057, 4.625854931368057) 0.03065235921522364
HMR_3455 : (-4.684257705058931, 4.684257705058931) 0.03065235921522364
HMR_3468 : (-1.3504972470841332, 1.3504972470841332) 0.03065235921522364
HMR_3469 : (-0.8718436485093178, 0.8718436485093178) 0.03065235921522364
HMR_1298 : (-1.3504972470841332, 1.3504972470841332) 0.03065235921522364
HMR_3355 : (-1.3504972470841332, 1.3504972470841332) 0.0306523592152236

In [34]:
all_and_simulation = get_simulator(model, envcond = media, constraints = one_gene_constrained_reactions) #one_gene_constrained_reactions
all_and_result = all_and_simulation.simulate()
print(all_and_result)

objective: 0.03065235921522364
Status: OPTIMAL
Constraints: OrderedDict([('HMR_9066', (-0.084, 1000)), ('HMR_9065', (-0.0626, 1000)), ('HMR_9063', (-0.876, 1000)), ('HMR_9067', (-0.03, 1000)), ('HMR_9038', (-0.042, 1000)), ('HMR_9039', (-0.105, 1000)), ('HMR_9040', (-0.105, 1000)), ('HMR_9041', (-0.146, 1000)), ('HMR_9042', (-0.03, 1000)), ('HMR_9043', (-0.066, 1000)), ('HMR_9069', (-0.042, 1000)), ('HMR_9044', (-0.095, 1000)), ('HMR_9045', (-0.016, 1000)), ('HMR_9064', (-0.10379, 1000)), ('HMR_9046', (-0.094, 1000)), ('HMR_9146', (-0.004, 1000)), ('HMR_9361', (-0.0072, 1000)), ('HMR_9378', (-0.004, 1000)), ('HMR_9145', (-0.004, 1000)), ('HMR_9144', (-0.004, 1000)), ('HMR_9143', (-0.0004, 1000)), ('HMR_9159', (-0.004, 1000)), ('HMR_9034', (-4.5, 1000)), ('EX_pail_hs[e]', (-1000, 1000)), ('HMR_7108', (0, 1000)), ('HMR_7110', (0, 1000)), ('HMR_7112', (0, 1000)), ('HMR_7114', (0, 1000)), ('HMR_7116', (0, 1000)), ('HMR_7118', (0, 1000)), ('HMR_7120', (0, 1000)), ('HMR_7122', (0, 1000)), ('

# 5. Observe and store modeling results

In [43]:
#save final constraints dictionary
_59m_constraints = one_gene_constrained_reactions
%store _59m_constraints

Stored '_59m_constraints' (dict)


In [36]:
all_and_result.find(['biomass_human'])

,Flux rate
Reaction ID,
biomass_human,0.030652


In [37]:
#find doubling time, by calculating 1/growth rate (g/gDW/hour)
1/(all_and_result.find(['biomass_human']))

,Flux rate
Reaction ID,
biomass_human,32.623916


In [38]:
#save results as final dataframe with reaction IDs and flux, for this cell line
ids = []
fluxes = []
for r in model.reactions:
    if 'EX' not in r.id:
        print(r.id, all_and_result.find([r.id]).iloc[0,0])
        ids.append(r.id)
        fluxes.append(all_and_result.find([r.id]).iloc[0,0])

HMR_3905 0.0
HMR_3907 0.0
HMR_4097 0.0
HMR_4099 0.0
HMR_4108 0.0
HMR_4133 0.0
HMR_4137 0.0
HMR_4281 15.298751058361592
HMR_4388 -14.022595991854311
HMR_4283 0.0
HMR_8357 0.0
HMR_4379 6.780146448568286
HMR_4301 0.0
HMR_4355 0.480781943074662
HMR_4358 0.0
HMR_4360 0.0
HMR_4363 10.795109121328665
HMR_4365 -10.795109121328665
HMR_4368 12.405779682585322
HMR_4370 0.3505983339818645
HMR_4371 2.0748581952784883e-05
HMR_4372 0.0
HMR_4373 -12.75639876514914
HMR_4375 -6.780016028910297
HMR_4377 0.0
HMR_4381 -7.261707945507745
HMR_4391 6.937958478720712
HMR_4394 0.0
HMR_4396 -0.01259156555000952
HMR_4521 0.0
HMR_6410 1.232660756790275
HMR_6412 1.232660756790275
HMR_7745 -3.196921733820329
HMR_7746 3.196921733820329
HMR_7747 0.0
HMR_7748 3.196921733820329
HMR_7749 0.0
HMR_8360 0.0
HMR_8652 0.0
HMR_8757 0.0
HMR_3989 0.0
HMR_4122 1.0157578796740812e-05
HMR_4837 0.0
HMR_5395 0.0
HMR_5396 0.0
HMR_9727 0.012450988313223844
HMR_5397 0.0
HMR_5398 0.0
HMR_5399 0.0
HMR_5400 0.0
HMR_5401 0.0
HMR_8568 0.0
HM

In [39]:
column_names = ['reaction', 'flux']
_59m_absolute_fluxes = pd.DataFrame(columns = column_names)
_59m_absolute_fluxes['reaction'] = ids
_59m_absolute_fluxes['flux'] = fluxes
_59m_absolute_fluxes

,reaction,flux
0,HMR_3905,0.0
1,HMR_3907,0.0
2,HMR_4097,0.0
3,HMR_4099,0.0
4,HMR_4108,0.0
...,...,...
11870,HMR_10128,0.0
11871,HMR_10129,0.0
11872,HMR_10130,0.0
11873,HMR_10131,0.0


In [40]:
%store _59m_absolute_fluxes

Stored '_59m_absolute_fluxes' (DataFrame)
